<a href="https://colab.research.google.com/github/akshats911/Movie-Recommendation-System/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#https://www.kaggle.com/tmdb/tmdb-movie-metadata
credits = pd.read_csv("tmdb_5000_credits.csv")
movies_df = pd.read_csv("tmdb_5000_movies.csv")

In [ ]:
credits.head()

In [ ]:
movies_df.head()

In [ ]:
print("Credits:",credits.shape)
print("Movies Dataframe:",movies_df.shape)

In [ ]:
credits_column_renamed = credits.rename(index=str, columns={"movie_id": "id"})
movies_df_merge = movies_df.merge(credits_column_renamed, on='id')
movies_df_merge.head()

In [ ]:
movies_cleaned_df = movies_df_merge.drop(columns=['homepage', 'title_x', 'title_y', 'status','production_countries'])
movies_cleaned_df.head()

In [ ]:
movies_cleaned_df.info()

using reviews for a content based system

In [ ]:
movies_cleaned_df.head(1)['overview']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
movies_cleaned_df['overview'] = movies_cleaned_df['overview'].fillna('')

In [ ]:
tfv_matrix = tfv.fit_transform(movies_cleaned_df['overview'])
tfv_matrix

In [ ]:
tfv_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel

# Computing the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)
sig[0]

In [ ]:
indices = pd.Series(movies_cleaned_df.index, index=movies_cleaned_df['original_title']).drop_duplicates()
indices

In [ ]:
sig[1234]

In [ ]:
list(enumerate(sig[indices['Spectre']]))

In [ ]:
sorted(list(enumerate(sig[indices['Spectre']])), key=lambda x: x[1], reverse=True)

In [ ]:
def give_rec(title, sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[1:11]
    movie_indices = [i[0] for i in sig_scores]

    return movies_cleaned_df['original_title'].iloc[movie_indices]

In [ ]:
give_rec('Avatar')